
### Baseline Recommendation Models
===============================

This notebook implements and evaluates baseline recommenders:
- Random recommender
- Popularity-based recommenders
- Time-decay popularity
- Trending items

In [ ]:
# ============================================================================
# Cell 1: Imports and Setup
# ============================================================================
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.data_loader import MovieLensLoader
from src.preprocess import prepare_data_for_training
from src.recommenders.popularity import (
    RandomRecommender,
    PopularityRecommender,
    TimeDecayPopularityRecommender,
    TrendingRecommender
)
from src.evaluation import RecommenderEvaluator
from src.utils import plot_model_comparison

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Imports successful")


In [ ]:
# ============================================================================
# Cell 2: Load and Prepare Data
# ============================================================================
print("Loading data...")
loader = MovieLensLoader()
ratings = loader.load_ratings()
movies = loader.load_movies()

# Prepare train/test split
print("\nPreparing train/test split...")
train, test, metadata = prepare_data_for_training(
    ratings,
    test_size=0.2,
    split_method='user_based',
    min_user_ratings=5,
    min_item_ratings=5,
    random_state=42
)

all_items = set(ratings['item_id'].unique())

print("\n✓ Data preparation complete")
print(f"  Train: {len(train):,} ratings")
print(f"  Test: {len(test):,} ratings")


In [ ]:
# ============================================================================
# Cell 3: Train Random Baseline
# ============================================================================
print("\n" + "="*70)
print("1. RANDOM RECOMMENDER (Baseline)")
print("="*70)

random_rec = RandomRecommender(random_state=42)
random_rec.fit(train)

# Test predictions
print("\nSample predictions:")
for user_id in [1, 10, 50]:
    for item_id in [1, 100, 500]:
        pred = random_rec.predict(user_id, item_id)
        print(f"  User {user_id}, Item {item_id}: {pred:.2f}")

# Generate recommendations
print("\nSample recommendations for user 1:")
recs = random_rec.recommend(user_id=1, n=10)
for i, (item_id, score) in enumerate(recs, 1):
    movie_title = movies[movies['item_id'] == item_id]['title'].values
    title = movie_title[0] if len(movie_title) > 0 else "Unknown"
    print(f"  {i}. {title} (score: {score:.3f})")



In [ ]:
# ============================================================================
# Cell 4: Train Popularity Recommenders
# ============================================================================
print("\n" + "="*70)
print("2. POPULARITY-BASED RECOMMENDERS")
print("="*70)

# Count-based popularity
print("\n2a. Popularity (Count-based)")
pop_count = PopularityRecommender(method='count')
pop_count.fit(train)

print("\nTop 10 most popular items:")
recs = pop_count.recommend(user_id=1, n=10, exclude_seen=False)
for i, (item_id, score) in enumerate(recs, 1):
    movie_title = movies[movies['item_id'] == item_id]['title'].values
    title = movie_title[0] if len(movie_title) > 0 else "Unknown"
    print(f"  {i}. {title} (score: {score:.3f})")

# Average rating popularity
print("\n2b. Popularity (Average Rating)")
pop_avg = PopularityRecommender(method='average')
pop_avg.fit(train)

# Weighted popularity
print("\n2c. Popularity (Weighted)")
pop_weighted = PopularityRecommender(method='weighted')
pop_weighted.fit(train)



In [ ]:
# ============================================================================
# Cell 5: Train Time-Decay Popularity
# ============================================================================
print("\n" + "="*70)
print("3. TIME-DECAY POPULARITY RECOMMENDER")
print("="*70)

time_decay_rec = TimeDecayPopularityRecommender(decay_rate=0.95, time_unit='days')
time_decay_rec.fit(train)

print("\nTop 10 recommendations (with time decay):")
recs = time_decay_rec.recommend(user_id=1, n=10, exclude_seen=False)
for i, (item_id, score) in enumerate(recs, 1):
    movie_title = movies[movies['item_id'] == item_id]['title'].values
    title = movie_title[0] if len(movie_title) > 0 else "Unknown"
    print(f"  {i}. {title} (score: {score:.3f})")


In [ ]:
# ============================================================================
# Cell 6: Train Trending Recommender
# ============================================================================
print("\n" + "="*70)
print("4. TRENDING RECOMMENDER")
print("="*70)

trending_rec = TrendingRecommender(window_days=30)
trending_rec.fit(train)

print("\nTop 10 trending items:")
recs = trending_rec.recommend(user_id=1, n=10, exclude_seen=False)
for i, (item_id, score) in enumerate(recs, 1):
    movie_title = movies[movies['item_id'] == item_id]['title'].values
    title = movie_title[0] if len(movie_title) > 0 else "Unknown"
    print(f"  {i}. {title} (score: {score:.3f})")


In [ ]:
# ============================================================================
# Cell 7: Evaluate All Baselines
# ============================================================================
print("\n" + "="*70)
print("COMPREHENSIVE EVALUATION")
print("="*70)

evaluator = RecommenderEvaluator(k_values=[5, 10, 20])

models = {
    'Random': random_rec,
    'Pop-Count': pop_count,
    'Pop-Average': pop_avg,
    'Pop-Weighted': pop_weighted,
    'Time-Decay': time_decay_rec,
    'Trending': trending_rec
}

results_df = evaluator.compare_models(
    models=models,
    test_data=test,
    train_data=train,
    all_items=all_items,
    n_recommendations=10
)

print("\n" + "="*70)
print("MODEL COMPARISON RESULTS")
print("="*70)
print(results_df)


In [ ]:
# ============================================================================
# Cell 8: Visualize Results
# ============================================================================
print("\n" + "="*70)
print("VISUALIZATION")
print("="*70)

# Select key metrics to plot
key_metrics = ['Precision@10', 'Recall@10', 'NDCG@10', 'Coverage', 'RMSE']
plot_model_comparison(results_df, metrics=key_metrics)

# Plot all metrics at different K values
precision_metrics = [col for col in results_df.columns if 'Precision@' in col]
recall_metrics = [col for col in results_df.columns if 'Recall@' in col]
ndcg_metrics = [col for col in results_df.columns if 'NDCG@' in col]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Precision@K
results_df[precision_metrics].T.plot(kind='line', ax=axes[0], marker='o', linewidth=2)
axes[0].set_title('Precision@K', fontsize=14, fontweight='bold')
axes[0].set_xlabel('K')
axes[0].set_ylabel('Precision')
axes[0].set_xticklabels([col.split('@')[1] for col in precision_metrics])
axes[0].legend(title='Model', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0].grid(alpha=0.3)

# Recall@K
results_df[recall_metrics].T.plot(kind='line', ax=axes[1], marker='o', linewidth=2)
axes[1].set_title('Recall@K', fontsize=14, fontweight='bold')
axes[1].set_xlabel('K')
axes[1].set_ylabel('Recall')
axes[1].set_xticklabels([col.split('@')[1] for col in recall_metrics])
axes[1].legend(title='Model', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1].grid(alpha=0.3)

# NDCG@K
results_df[ndcg_metrics].T.plot(kind='line', ax=axes[2], marker='o', linewidth=2)
axes[2].set_title('NDCG@K', fontsize=14, fontweight='bold')
axes[2].set_xlabel('K')
axes[2].set_ylabel('NDCG')
axes[2].set_xticklabels([col.split('@')[1] for col in ndcg_metrics])
axes[2].legend(title='Model', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()




In [ ]:
# ============================================================================
# Cell 9: Analyze Coverage and Diversity
# ============================================================================
print("\n" + "="*70)
print("COVERAGE AND DIVERSITY ANALYSIS")
print("="*70)

# Coverage comparison
print("\nCatalog Coverage:")
for model_name, model in models.items():
    coverage = results_df.loc[model_name, 'Coverage']
    print(f"  {model_name:15s}: {coverage:.4f} ({coverage*len(all_items):.0f}/{len(all_items)} items)")

# Visualize coverage
plt.figure(figsize=(10, 6))
coverage_data = results_df['Coverage'].sort_values(ascending=False)
coverage_data.plot(kind='barh', color='teal', edgecolor='black')
plt.title('Catalog Coverage by Model', fontsize=14, fontweight='bold')
plt.xlabel('Coverage')
plt.xlim(0, 1)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# ============================================================================
# Cell 10: Recommendation Overlap Analysis
# ============================================================================
print("\n" + "="*70)
print("RECOMMENDATION OVERLAP ANALYSIS")
print("="*70)

# Get recommendations from all models for a sample user
sample_user = 1
all_recs = {}

for model_name, model in models.items():
    recs = model.recommend(user_id=sample_user, n=20, exclude_seen=True)
    all_recs[model_name] = set([item_id for item_id, _ in recs])

# Calculate overlap matrix
overlap_matrix = pd.DataFrame(
    index=models.keys(),
    columns=models.keys(),
    dtype=float
)

for model1 in models.keys():
    for model2 in models.keys():
        if model1 in all_recs and model2 in all_recs:
            overlap = len(all_recs[model1] & all_recs[model2])
            overlap_matrix.loc[model1, model2] = overlap / 20.0

print(f"\nRecommendation Overlap for User {sample_user} (Top-20):")
print(overlap_matrix)

# Visualize overlap
plt.figure(figsize=(10, 8))
sns.heatmap(
    overlap_matrix.astype(float),
    annot=True,
    fmt='.2f',
    cmap='YlOrRd',
    cbar_kws={'label': 'Overlap Ratio'},
    square=True
)
plt.title('Recommendation Overlap Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


In [ ]:
# ============================================================================
# Cell 11: Key Insights
# ============================================================================
print("\n" + "="*70)
print("KEY INSIGHTS FROM BASELINE MODELS")
print("="*70)

best_precision = results_df['Precision@10'].idxmax()
best_recall = results_df['Recall@10'].idxmax()
best_ndcg = results_df['NDCG@10'].idxmax()
best_coverage = results_df['Coverage'].idxmax()
best_rmse = results_df['RMSE'].idxmin()

insights = f"""
🎯 PERFORMANCE SUMMARY:
   • Best Precision@10: {best_precision} ({results_df.loc[best_precision, 'Precision@10']:.4f})
   • Best Recall@10: {best_recall} ({results_df.loc[best_recall, 'Recall@10']:.4f})
   • Best NDCG@10: {best_ndcg} ({results_df.loc[best_ndcg, 'NDCG@10']:.4f})
   • Best Coverage: {best_coverage} ({results_df.loc[best_coverage, 'Coverage']:.4f})
   • Best RMSE: {best_rmse} ({results_df.loc[best_rmse, 'RMSE']:.4f})

💡 OBSERVATIONS:
   • Random baseline performs poorly (as expected)
   • Popularity-based methods show reasonable performance
   • Weighted popularity balances count and quality
   • Time-decay and trending capture temporal dynamics
   • All baselines have similar RMSE (predicting average)

⚠️ LIMITATIONS:
   • All baselines recommend same items to all users (no personalization)
   • High coverage but low precision/recall
   • Cannot capture user-specific preferences
   • Suffer from popularity bias

🚀 NEXT STEPS:
   • Implement collaborative filtering for personalization
   • Try content-based methods for cold-start items
   • Combine approaches in hybrid models
"""

print(insights)


In [ ]:


# ============================================================================
# Cell 12: Save Results
# ============================================================================
print("\n" + "="*70)
print("SAVING RESULTS")
print("="*70)

# Save evaluation results
results_df.to_csv('../data/processed/baseline_results.csv')
print("✓ Results saved to data/processed/baseline_results.csv")

# Save best models
import pickle
os.makedirs('../models', exist_ok=True)

pop_weighted.save('../models/popularity_weighted.pkl')
time_decay_rec.save('../models/time_decay.pkl')

print("✓ Best models saved to models/")
print("\n✅ Baseline evaluation complete!")